In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import joblib

from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from rdkit import Chem
from rdkit.Chem import Descriptors

import sys
sys.path.append('../')


In [2]:
def count_cf_bonds(mol):
    if mol is None:
        return 0
    try:
        abstract_cf = Chem.MolFromSmarts('C-F')
        if abstract_cf is None:
            return 0
        cf_bonds = mol.GetSubstructMatches(abstract_cf)
        return len(cf_bonds)
    except:
        return 0


In [3]:
# read data 
ldtoxdb = pd.read_csv('ldtoxdb-mordred2.csv')
pfas8k = pd.read_csv('pfas8k-mordred.csv')

In [4]:
# Covert SMILES to RDKit mol objects
ldtoxdb['rd_mol'] = ldtoxdb.SMI.apply(Chem.MolFromSmiles)
# drop rows with no mol object(non mol object can not be appplied molwt)
ldtoxdb = ldtoxdb.dropna(subset=['rd_mol'])
# count cf bonds
ldtoxdb['n_cf_bonds'] = ldtoxdb.rd_mol.apply(count_cf_bonds)
# calculate molecular weight (for coversion to mg/kg)   
ldtoxdb['mol_wt'] = ldtoxdb.rd_mol.apply(Descriptors.MolWt)
# classify pfas like molecules
ldtoxdb['is_pfas_like'] = ldtoxdb['n_cf_bonds'] >= 22

pfas8k['mol'] = pfas8k.SMILES.apply(Chem.MolFromSmiles)
pfas8k = pfas8k.dropna(subset=['mol'])
pfas8k['canon_smi'] = pfas8k['mol'].apply(Chem.MolToSmiles)
pfas8k = pfas8k.drop('mol', axis=1)
ldtoxdb['is_pfas'] = ldtoxdb.SMI.isin(pfas8k.canon_smi)

[22:28:12] Explicit valence for atom # 1 Cl, 3, is greater than permitted


In [5]:
ldtoxdb.shape,pfas8k.shape

((13329, 490), (8154, 357))

In [6]:
class LD50UnitConverter():
    # covert neglogld50 to mg/kg
    def convert_to_mgkg(self, neglogld50s, smiles):

        for neglogld50, smile in zip(neglogld50s, smiles):
            molwt = Descriptors.MolWt(Chem.MolFromSmiles(smile[0]))
            yield (10**(-1*neglogld50[0]))*1000*molwt
        
            

    # covert mg/kg to epa classes
    def convert_to_epa(self, neglogld50s, smiles):
        mgkg = list(self.convert_to_mgkg(neglogld50s=neglogld50s, smiles=smiles))

        return pd.cut(mgkg, labels=(0,1,2,3), bins=(-np.inf,50,500,5000, np.inf))

In [7]:
def data_loader(dataframe):
    # feature extraction
    mordred = dataframe.columns[6:-5]
    
    X = dataframe[mordred] 
    # target
    y = dataframe['NeglogLD50']  
    smiles = dataframe['SMI']  
    print(f'Data loaded with {X.shape[1]} features and {X.shape[0]} samples')
    return X, y, smiles

In [8]:
class RandomForest:
    def __init__(self, n_estimators, max_depth, min_samples_split, min_samples_leaf, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.model = RandomForestRegressor(n_estimators=self.n_estimators, 
                                            max_depth=self.max_depth, 
                                            min_samples_split=self.min_samples_split, 
                                            min_samples_leaf=self.min_samples_leaf,
                                            random_state=self.random_state,
                                            n_jobs=-1)
        self.scaler = StandardScaler()
        
        if not os.path.exists('chkpts'):
            os.makedirs('chkpts')   
        
        
    def fit(self, X, y):
        y_scaled = self.scaler.fit_transform(y.values.reshape(-1, 1)).ravel()
        self.model.fit(X, y_scaled)

    def predict(self, X):
        y_pred_scaled = self.model.predict(X)
        return self.scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
    
    
    def train_and_evaluate(self, X, y, smiles, n_splits=5, converter=None):
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=self.random_state)
        
        fold_predictions = []
        
        # training and prediction for each fold
        for fold, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            smiles_train, smiles_test = smiles.iloc[train_index], smiles.iloc[test_index]
            
            self.fit(X_train, y_train)
            
            y_pred = self.predict(X_test)
            
            fn = f'chkpts/estimator_fold_{fold+1}.joblib'
            joblib.dump(self.model, fn)
            
            results = pd.DataFrame({
                'fold': fold,
                'smiles': smiles_test,
                'prediction_neglogld50': y_pred,
                'actual_neglogld50': y_test,
                'index': test_index
            })
            
            if converter:
                results['prediction_mgkg'] = list(converter.convert_to_mgkg(y_pred.reshape(-1, 1), smiles_test.values.reshape(-1, 1)))
                results['prediction_epa'] = converter.convert_to_epa(y_pred.reshape(-1, 1), smiles_test.values.reshape(-1, 1))
                results['actual_mgkg'] = list(converter.convert_to_mgkg(y_test.values.reshape(-1, 1), smiles_test.values.reshape(-1, 1)))
                results['actual_epa'] = converter.convert_to_epa(y_test.values.reshape(-1, 1), smiles_test.values.reshape(-1, 1))
            
            fold_predictions.append(results)
            print(f"Fold {fold+1} training complete")
            print('-'*10)
        
        all_predictions = pd.concat(fold_predictions).sort_values('index').reset_index(drop=True)
        all_predictions.to_csv('rf_predictions.csv', index=False)
        
        print("Predictions saved to rf_predictions.csv")
        
        return all_predictions



In [9]:
rf = RandomForest(n_estimators=4096, max_depth=32, min_samples_split=2, min_samples_leaf=1, random_state=42)
X, y, smiles = data_loader(ldtoxdb)
converter = LD50UnitConverter()
predictions = rf.train_and_evaluate(X, y, smiles, converter=converter)

Data loaded with 479 features and 13329 samples
Fold 1 training complete
----------
Fold 2 training complete
----------
Fold 3 training complete
----------
Fold 4 training complete
----------
Fold 5 training complete
----------
Predictions saved to rf_predictions.csv


In [10]:
def calculate_scores(predictions_df):

    scores = {}
    
    # Calculate R2 score
    scores['R2'] = r2_score(predictions_df['actual_neglogld50'], predictions_df['prediction_neglogld50'])
    
    # Calculate Mean Absolute Error (MAE)
    scores['MAE'] = mean_absolute_error(predictions_df['actual_neglogld50'], predictions_df['prediction_neglogld50'])
    
    # Calculate Root Mean Squared Error (RMSE)
    scores['RMSE'] = np.sqrt(mean_squared_error(predictions_df['actual_neglogld50'], predictions_df['prediction_neglogld50']))
    
    # Calculate EPA classification accuracy if available
    if 'actual_epa' in predictions_df.columns and 'prediction_epa' in predictions_df.columns:
        scores['EPA_Accuracy'] = (predictions_df['actual_epa'] == predictions_df['prediction_epa']).mean()
    
    return scores

In [11]:
# Calculate scores
scores = calculate_scores(predictions)

for metric, value in scores.items():
    print(f"{metric}: {value}")


R2: 0.6507597734775254
MAE: 0.3705976329989629
RMSE: 0.5209214928014738
EPA_Accuracy: 0.6539875459524346
